In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.io.compression.codec", "zstd") \
    .config("spark.sql.execution.pythonUDF.arrow.enabled", "true") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sol.shuffle.part-tions", 200) \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "5") \
    .config("spark.sql.files.maxRecordsPerFile", 10_000) \
    .config("spark.debug.maxToStringFields", 1000) \
    .config("spark.executor.heartbeatInterval", "3600s") \
    .config("spark.network.timeout", "7200s") \
    .config("spark.network.timeoutInterval", "3600s") \
    .enableHiveSupport() \
    .getOrCreate()
 
sc = spark.sparkContext

 # Import all necessary libraries
 

In [31]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from geopy.geocoders import OpenCage
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from opencage.geocoder import OpenCageGeocode
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from opencage.geocoder import OpenCageGeocode
from decimal import Decimal
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Start spark session

In [4]:
spark = SparkSession.builder.appName("SparkETL").getOrCreate()


# Read the restaurants_csv folder

In [40]:
#Прежний метод очень доллго грузился по этому использовался для проверки нагрузки
%load_ext memory_profiler
%memit

folder_path = "restaurant_csv/"

# Получаю список файлов в папке с ресторанами
files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".csv")]

# Считайте все CSV-файлы в DataFrame
restaurant_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(files)

#Cхему DataFrame
restaurant_df.printSchema()

#DataFrame
restaurant_df.show()


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 69.37 MiB, increment: 0.13 MiB
root
 |-- id: long (nullable = true)
 |-- franchise_id: integer (nullable = true)
 |-- franchise_name: string (nullable = true)
 |-- restaurant_franchise_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)

+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|          id|franchise_id|      franchise_name|restaurant_franchise_id|country|          city|   lat|    lng|
+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|197568495625|          10|    The Golden Spoon|                  24784|     US|       Decatur|34.578|-87.021|
| 17179869242|          59|         Azalea Cafe|                  10902|     FR|    

In [4]:

api_key = '8ed618e56d764009b0105f340b74bce5'
geolocator = OpenCage(api_key)

# Filter out records with null latitude or longitude

In [41]:
# Filter out records with null latitude or longitude
filtered_df = restaurant_df.filter(restaurant_df["lat"].isNotNull() & restaurant_df["lng"].isNotNull())


In [42]:
filtered_df.show()

+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|          id|franchise_id|      franchise_name|restaurant_franchise_id|country|          city|   lat|    lng|
+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|197568495625|          10|    The Golden Spoon|                  24784|     US|       Decatur|34.578|-87.021|
| 17179869242|          59|         Azalea Cafe|                  10902|     FR|         Paris|48.861|  2.368|
|214748364826|          27|     The Corner Cafe|                  92040|     US|    Rapid City| 44.08|-103.25|
|154618822706|          51|        The Pizzeria|                  41484|     AT|        Vienna|48.213| 16.413|
|163208757312|          65|       Chef's Corner|                  96638|     GB|        London|51.495| -0.191|
| 68719476763|          28|    The Spicy Pickle|                  77517|     US|      Grayling|44.657|-84.744|
|

In [17]:
#Cхему DataFrame
filtered_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- franchise_id: integer (nullable = true)
 |-- franchise_name: string (nullable = true)
 |-- restaurant_franchise_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)



# Check lat and lng for incorrect values using the OpenCage Geocoding API 

In [45]:

# Define the reverse_geocode_udf function
@udf(StringType())
def reverse_geocode_udf(lat, lon):
    try:
        # Convert 'lat' and 'lon' to float
        lat_float = float(lat)
        lon_float = float(lon)

        # Perform reverse geocoding using OpenCageGeocode API
        result = geocoder.reverse_geocode(lat_float, lon_float)

        if result and len(result):
            return result[0]['formatted']
        else:
            return None
    except ValueError:
        # Handle the case where conversion to float fails
        return None

# Use withColumn to add a new column 'address'
result_df = filtered_df.withColumn("address", reverse_geocode_udf(filtered_df['lat'], filtered_df['lng']))

# Show the resulting DataFrame
result_df.show(truncate=False)

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Spark\spark-3.5.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 1100, in main
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 9) than that in driver 3.7, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.


 # Im really trying hard but that error doesn't handle
 